## 1. Import Dependencies

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

import numpy as np
import random
import os
import pandas as pd


## 2. Creat Env

In [2]:
class CustomEnv(Env):
    def __init__(self, data_path = "Data_test\data_demo.csv"):
        
        self.action_space = Discrete(141)
        self.observation_space = Discrete(242, start=100)
        #self.observation_space = Box(low=100, high=161, shape=(1,), dtype=int)
        
        data = pd.read_csv("Data_test\data_demo.csv")
        #self.data_request = data.request
        #self.data_response = data.response
        #self.data_lable = data.lable
        self.data_request_list = data.request.to_list()
        self.data_general_request_list = data.general_request.to_list()
        self.data_general_unique_list = data.general_unique.tolist()
        self.data_response_list = data.response_simple.to_list()
        self.data_lable_list = data.lable.to_list()
        self.data_value_list = data._value_.to_list()
        self.data_key_request_list = data.key_request.to_list()
                 
        #Lấy index các dạng tổng quát
        self.lable_exploit_list = [self.data_value_list[index] for index, label in enumerate(self.data_lable_list) if label == 'exploit']
        self.lable_upload_list = [self.data_value_list[index] for index, label in enumerate(self.data_lable_list) if label == 'upload']
        self.lable_end_list = [self.data_value_list[index] for index, label in enumerate(self.data_lable_list) if label == 'end']             

        self.general_upload = [index for index, general in enumerate(self.data_general_unique_list) if general == 'MixString-MixStringMixString-MixStringname=PureStringfilename=PureString.php']
        self.general_end = [index for index, general in enumerate(self.data_general_unique_list) if general == 'MixString']

        self.general_cmdinj_1_collect = [index for index, general in enumerate(self.data_general_unique_list) if general =="ip=Number.Number.Number.Number%HexString%HexStringMixString&Submit=PureString"]
        self.general_cmdinj_2_collect = [index for index, general in enumerate(self.data_general_unique_list) if general =="ip=Number.Number.Number.Number%HexString%HexStringMixString-MixString&Submit=PureString"]
        self.general_cmdinj_3_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="ip=Number.Number.Number.Number%HexString%HexStringMixString-MixString%HexStringMixString%HexStringMixStringNumber.Number.Number.NumberNumber&Submit=PureString"]
        self.general_cmdinj_4_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="ip=Number.Number.Number.Number%HexString%HexStringMixString-MixString%HexString%HexString%HexStringMixString%HexStringMixString%HexStringNumber.Number.Number.Number%HexStringNumberNumber%HexString%HexStringNumber&Submit=PureString"]
        
        self.general_sql_1_collect = [index for index, general in enumerate(self.data_general_unique_list) if general =="?id=%HexStringNumber&Submit=PureString"]
        self.general_sql_2_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?id=%HexString%HexStringNumberMixString%HexStringNumber%HexString%HexString%HexStringNumber&Submit=PureString"]
        self.general_sql_3_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?id=%HexStringNumber%HexStringMixStringMixStringMixString%HexStringMixStringMixStringMixString%HexString&Submit=PureString"]
        self.general_sql_4_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="id=NumberMixStringNumber%HexStringNumber%HexString&Submit=PureString"]
        self.general_sql_5_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="id=%HexStringNumber%HexStringMixStringMixStringMixString%HexStringMixStringMixStringMixString%HexString&Submit=PureString"]
        self.general_sql_6_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?id=Number%HexStringMixStringMixString%HexStringNumber%HexString%HexString&Submit=PureString"]
        self.general_sql_7_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="id=NumberMixStringMixString%HexStringNumber%HexString%HexString&Submit=PureString"]
        
        
        self.general_xss_1_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?default=PureString%HexStringMixString%HexStringMixString(%HexStringMixString%HexStringMixString%HexStringMixString%HexStringNumber)%HexString/PathString%HexString"]
        self.general_xss_2_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?default=PureString%HexString/PathString%HexString%HexString/PathString%HexString%HexStringMixString%HexStringsrc=%HexStringMixString%HexString%HexStringonerror=%HexStringMixString(%HexStringNumberMixString.MixString%HexString)%HexString%HexString"]
        self.general_xss_3_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?name=%HexStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString%HexString%HexStringMixString%HexString"]
        self.general_xss_4_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?name=%HexStringMixStringMixString%HexStringMixStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString"]
        self.general_xss_5_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="txtName=PureString&mtxMessage=%HexStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString%HexString%HexStringMixString%HexString&btnSign=PureStringMixString"]
        self.general_xss_6_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="txtName=%HexStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString%HexString%HexStringMixString%HexString&mtxMessage=PureString&btnSign=PureStringMixString"]
        self.general_xss_7_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="txtName=%HexStringMixStringMixString%HexStringMixStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString&mtxMessage=PureString&btnSign=PureStringMixString"]
        
        self.general_csrf_1_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?password_new=NumberPureString&password_conf=PureString&Change=PureString"]
        
        self.general_path_raversal_1_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?page=/PathString/PathString/PathString/PathString/PathString/PathString/PathString/PathString"]
        self.general_path_raversal_2_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?page=PureString.php%HexString/PathString/PathString/PathString/PathString/PathString/PathString/PathString/PathString"]                       
        
        
        # Initialize state
        self.lable_collect_list = [self.data_value_list[index] for index, label in enumerate(self.data_lable_list) if label == 'collect']
        self.state = np.random.choice(self.lable_collect_list)
        
        # Set shower length
        self.shower_length = 10
        
    def step(self, action):
        self.total_reward = 0
        self.shower_length -= 1 
        # Calculate reward
        if self.state in self.lable_collect_list:
            if action in self.general_cmdinj_1_collect and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 0
            elif action in self.general_cmdinj_2_collect and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 0
            elif action in self.general_sql_1_collect and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 0
            else:
                reward = -1
                
        elif self.state in self.lable_exploit_list:
            if action in self.general_cmdinj_3_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_cmdinj_4_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_2_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_3_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_4_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_5_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_6_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_7_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            
            elif action in self.general_xss_1_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_2_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_3_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_4_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_5_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_6_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_7_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
                
            elif action in self.general_csrf_1_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
                
            elif action in self.general_path_raversal_1_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_path_raversal_2_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            else:
                reward = -1
            
            
        elif self.state in self.lable_upload_list:
            if action in self.general_upload and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 5
            else:
                reward = -1
        elif self.state in self.lable_end_list and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
            if self.total_reward > 0:
                reward = 0
            else:
                reward = -1
        elif self.state in self.lable_end_list and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
            if self.total_reward <= 0:
                reward = -1
        else:
            reward = -1        
            
       
            
        self.total_reward += reward
            
       
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        if done:
            self.state = np.random.choice(self.lable_collect_list)
        else:
            self.state = np.random.choice(self.data_value_list)
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    
    def reset(self):
        # Reset shower temperature
        self.state = np.random.choice(self.lable_collect_list)
        # Reset shower time
        self.shower_length = 10
        return self.state

In [3]:
env1 = CustomEnv()

## 3.Reload Model and Test

In [4]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_Test_env10_model_200k_esp')

In [5]:
PPO_path

'Training\\Saved Models\\PPO_Test_env10_model_200k_esp'

In [6]:
model = PPO.load(PPO_path, env1)

C:\Users\Asus\anaconda3\Lib\site-packages\torch\cuda\__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
C:\Users\Asus\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## Input_Function

In [17]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import re

#Ham đưa request về tổng quát 
def request_general(input_string):
    # pattern = re.compile(r'\?|&|=|-|.php|%[0-9A-Fa-f]{2}|\w+|\.|\(|\)')
    pattern = re.compile(r'/\.\.|/[a-zA-Z]+|\?|&|=|-|.php|%[0-9A-Fa-f]{2}|\w+|\.|\(|\)')
    matches = pattern.findall(input_string)
    #print(matches)
    print("\n")

    #Đưa về dạng tổng quát
    arr =[]
    result_arr = []
    Number_arr = []
    Hex_arr = []
    Pure_arr = []
    Mix_arr = []
    
    result_str = ""

    arr = matches
    for index in range(len(arr)):
        current_index = index
        if arr[index].isdigit():
            Number_arr.append(arr[index])
            arr[index] = "Number"
            result_arr.append(arr[index])
        elif "%" in arr[index]:
            arr[index] = "%HexString"
            result_arr.append(arr[index])
        elif ((index+1) < len(arr)) and arr[index+1] == "=":
            result_arr.append(arr[index])
        elif arr[index-1] == "=" and arr[index].isalnum():
            arr[index] = "PureString"
            result_arr.append(arr[index])
        elif arr[index].isalnum():
            Mix_arr.append(arr[index])
            arr[index] = "MixString"
            result_arr.append(arr[index])
        elif "/" in arr[index]:
            arr[index] = "/PathString"
            result_arr.append(arr[index])
    #     elif arr[index-1] == "-" and arr[index].isalnum():
    #         result_arr.append(arr[index])
        else:
            result_arr.append(arr[index])

    #print(result_arr,"\n")
    result_Mix_Num = Mix_arr + Number_arr
    result_key = ' '.join(result_Mix_Num)
    #print(result_key)

    for char in result_arr:
        result_str += char
    #print(result_str)
    return result_str
    #return result_key
    
#Hàm lấy ra key của request
def request_general_key(input_string):
    # pattern = re.compile(r'\?|&|=|-|.php|%[0-9A-Fa-f]{2}|\w+|\.|\(|\)')
    pattern = re.compile(r'/\.\.|/[a-zA-Z]+|\?|&|=|-|.php|%[0-9A-Fa-f]{2}|\w+|\.|\(|\)')
    matches = pattern.findall(input_string)
    #print(matches)
    print("\n")

    #Đưa về dạng tổng quát
    arr =[]
    result_arr = []
    Number_arr = []
    Hex_arr = []
    Pure_arr = []
    Mix_arr = []
    
    result_str = ""

    arr = matches
    for index in range(len(arr)):
        current_index = index
        if arr[index].isdigit():
            Number_arr.append(arr[index])
            arr[index] = "Number"
            result_arr.append(arr[index])
        elif "%" in arr[index]:
            arr[index] = "%HexString"
            result_arr.append(arr[index])
        elif ((index+1) < len(arr)) and arr[index+1] == "=":
            result_arr.append(arr[index])
        elif arr[index-1] == "=" and arr[index].isalnum():
            arr[index] = "PureString"
            result_arr.append(arr[index])
        elif arr[index].isalnum():
            Mix_arr.append(arr[index])
            arr[index] = "MixString"
            result_arr.append(arr[index])
        elif "/" in arr[index]:
            arr[index] = "/PathString"
            result_arr.append(arr[index])
    #     elif arr[index-1] == "-" and arr[index].isalnum():
    #         result_arr.append(arr[index])
        else:
            result_arr.append(arr[index])

    #print(result_arr,"\n")
    result_Mix_Num = Mix_arr + Number_arr
    result_key = ' '.join(result_Mix_Num)
    #print(result_key)

    for char in result_arr:
        result_str += char
    #print(result_str)
    #return result_str
    return result_key

# tách request ra từng chuỗi
def request_general_arr(input_string):
    # pattern = re.compile(r'\?|&|=|-|.php|%[0-9A-Fa-f]{2}|\w+|\.|\(|\)')
    pattern = re.compile(r'/\.\.|/[a-zA-Z]+|\?|&|=|-|.php|%[0-9A-Fa-f]{2}|\w+|\.|\(|\)')
    matches = pattern.findall(input_string)
    #print(matches)
    print("\n")

    #Đưa về dạng tổng quát
    arr =[]
    result_arr = []
    Number_arr = []
    Hex_arr = []
    Pure_arr = []
    Mix_arr = []
    
    result_str = ""

    arr = matches
    for index in range(len(arr)):
        current_index = index
        if arr[index].isdigit():
            Number_arr.append(arr[index])
            arr[index] = "Number"
            result_arr.append(arr[index])
        elif "%" in arr[index]:
            arr[index] = "%HexString"
            result_arr.append(arr[index])
        elif ((index+1) < len(arr)) and arr[index+1] == "=":
            result_arr.append(arr[index])
        elif arr[index-1] == "=" and arr[index].isalnum():
            arr[index] = "PureString"
            result_arr.append(arr[index])
        elif arr[index].isalnum():
            Mix_arr.append(arr[index])
            arr[index] = "MixString"
            result_arr.append(arr[index])
        elif "/" in arr[index]:
            arr[index] = "/PathString"
            result_arr.append(arr[index])
    #     elif arr[index-1] == "-" and arr[index].isalnum():
    #         result_arr.append(arr[index])
        else:
            result_arr.append(arr[index])

    #print(result_arr,"\n")
    result_Mix_Num = Mix_arr + Number_arr
    result_key = ' '.join(result_Mix_Num)
    #print(result_key)

    for char in result_arr:
        result_str += char
    #print(result_str)
    #return result_str
    #return result_key
    return result_arr

#Hàm nhập request
def _input_():
    input_request = input()
    input_general = request_general(input_request)
    input_general_arr = request_general_arr(input_request)
    input_general_key = request_general_key(input_request)
    
    if input_general in env1.data_general_request_list and input_request in env1.data_request_list:
        init_state = env1.data_value_list[env1.data_request_list.index(input_request)]
        init_state_np = np.array([init_state])
    elif input_general in env1.data_general_request_list and input_request not in env1.data_request_list:
        #all_value_input_general = [env1.data_value_list[index] for index, general in enumerate(env1.data_general_request_list) if general == input_general]
        all_value_input_general_key = [env1.data_value_list[index] for index, general_key in enumerate(env1.data_key_request_list) if general_key == input_general_key]
        init_state = random.choice(all_value_input_general_key)
        init_state_np = np.array([init_state])
    else:
        #tách ra từng chuỗi để so sánh "/PathString", "%HexString" có trong request ko
        if "/PathString" in input_general_arr:
            init_state = 404
        elif "%HexString" in input_general_arr:
            init_state = 400
        else:
            init_state = 2000
        init_state_np = np.array([init_state])
        
    return init_state_np


    
    

## Enter_input

In [138]:
init_state = np.array(_input_())

?page=/../../../../etc/passwd








In [139]:
print(init_state)

[404]


## If input_general in DB --> agent response

## Test 1

In [140]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

404 not found


## Test 2

In [75]:
init_state = np.array(_input_())

ip=127.0.0.1%3B+ls&Submit=Submit








In [76]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

Request [137]: ip=127.0.0.1%3B+ls&Submit=Submit
Request general [137]: ip=Number.Number.Number.Number%HexStringMixString&Submit=PureString
Selected Action [49]: 
help
index.php
source


## Test 3

In [77]:
init_state = np.array(_input_())

?id=%271&Submit=Submit








In [78]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

Request [157]: ?id=%271&Submit=Submit
Request general [157]: ?id=%HexStringNumber&Submit=PureString
Selected Action [57]: 
<pre>You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '1'' at line 1</pre>


## Test 4

In [79]:
init_state = np.array(_input_())

id=0+OR+0%3D0%23&Submit=Submit








In [80]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

Request [168]: id=0+OR+0%3D0%23&Submit=Submit
Request general [168]: id=NumberMixStringNumber%HexStringNumber%HexString&Submit=PureString
Selected Action [68]: 
ID: 0 OR 0=0#
First name: admin
Surname: admin
ID: 0 OR 0=0#
First name: Gordon
Surname: Brown
ID: 0 OR 0=0#
First name: Hack
Surname: Me
ID: 0 OR 0=0#
First name: Pablo
Surname: Picasso
ID: 0 OR 0=0#
First name: Bob
Surname: Smith


## Test 5

In [81]:
init_state = np.array(_input_())

id=1%27+UNION+SELECT+user%2C+password+FROM+users%23&Submit=Submit








In [82]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

Request [174]: id=1%27+UNION+SELECT+user%2C+password+FROM+users%23&Submit=Submit
Request general [174]: id=Number%HexStringMixStringMixStringMixString%HexStringMixStringMixStringMixString%HexString&Submit=PureString
Selected Action [74]: 
ID: 1' UNION SELECT user, password FROM users#
First name: admin
Surname: admin
ID: 1' UNION SELECT user, password FROM users#
First name: admin
Surname: 5f4dcc3b5aa765d61d8327deb882cf99
ID: 1' UNION SELECT user, password FROM users#
First name: gordonb
Surname: e99a18c428cb38d5f260853678922e03
ID: 1' UNION SELECT user, password FROM users#
First name: 1337
Surname: 8d3533d75ae2c3966d7e0d4fcc69216b
ID: 1' UNION SELECT user, password FROM users#
First name: pablo
Surname: 0d107d09f5bbe40cade3de5c71e9e9b7
ID: 1' UNION SELECT user, password FROM users#
First name: smithy
Surname: 5f4dcc3b5aa765d61d8327deb882cf99


## Test 6

In [83]:
init_state = np.array(_input_())

?name=%3Cscript%3Ealert%28document.cookie%29%3B%3C%2Fscript%3E








In [84]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

Request [203]: ?name=%3Cscript%3Ealert%28document.cookie%29%3B%3C%2Fscript%3E
Request general [203]: ?name=%HexStringMixString%HexStringMixString%HexStringMixString.MixString%HexString%HexString%HexString%HexStringMixString%HexString
Selected Action [103]: 
PHPSESSID=ni55hnpu8lnn1p3r75daalv313; security=low


## Test 7

In [96]:
init_state = np.array(_input_())

?default=French%3Cscript%3Ealert(%22you%20been%20hacked!%22)%3C/script%3E 








In [97]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

Request [192]: ?default=French%3Cscript%3Ealert(%22you%20been%20hacked!%22)%3C/script%3E 
Request general [192]: ?default=PureString%HexStringMixString%HexStringMixString(%HexStringMixString%HexStringMixString%HexStringMixString%HexString)%HexString/PathString%HexString
Selected Action [46]: 
Linux 858774c0c3a7 6.1.0-kali5-amd64 #1 SMP PREEMPT_DYNAMIC Debian 6.1.12-1kali2 (2023-02-23) x86_64 GNU/Linux


## Test 8

In [90]:
init_state = np.array(_input_())

txtName=%3Cimg+src%3Dx+onerror%3Dalert%28%22you_hacked%21%22%29%3E&mtxMessage=uit&btnSign=Sign+Guestbook








In [91]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

Request [225]: txtName=%3Cimg+src%3Dx+onerror%3Dalert%28%22you_hacked%21%22%29%3E&mtxMessage=uit&btnSign=Sign+Guestbook
Request general [225]: txtName=%HexStringMixStringMixString%HexStringMixStringMixString%HexStringMixString%HexString%HexStringyou_hacked%HexString%HexString%HexString%HexString&mtxMessage=PureString&btnSign=PureStringMixString
Selected Action [125]: 
you_hacked!


## Test 9

In [92]:
init_state = np.array(_input_())

?page=file1.php%0C/../../../../../../etc/passwd








In [93]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

Request [233]: ?page=file1.php%0C/../../../../../../etc/passwd
Request general [233]: ?page=PureString.php%HexString/PathString/PathString/PathString/PathString/PathString/PathString/PathString/PathString
Selected Action [133]: 
root:x:0:0:root:/root:/bin/bash daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin bin:x:2:2:bin:/bin:/usr/sbin/nologin sys:x:3:3:sys:/dev:/usr/sbin/nologin sync:x:4:65534:sync:/bin:/bin/sync games:x:5:60:games:/usr/games:/usr/sbin/nologin man:x:6:12:man:/var/cache/man:/usr/sbin/nologin lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin mail:x:8:8:mail:/var/mail:/usr/sbin/nologin news:x:9:9:news:/var/spool/news:/usr/sbin/nologin uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin proxy:x:13:13:proxy:/bin:/usr/sbin/nologin www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin backup:x:34:34:backup:/var/backups:/usr/sbin/nologin list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin gnats:x:41:41:Gnats Bug-Reporting S

## Test 10

In [94]:
init_state = np.array(_input_())

Content-Disposition: form-data; name="uploaded"; filename="shell.php"








In [95]:
if 100<= init_state <=241:
    obs = init_state
    print(f"Request {obs}: {env1.data_request_list[int(env1.data_value_list.index(obs))]}")
    print(f"Request general {obs}: {env1.data_general_request_list[int(env1.data_value_list.index(obs))]}")

    action, _ = model.predict(obs)
    print(f"Selected Action {action}: \n{env1.data_response_list[int(action)]}")
else:
    if init_state == 404:
        print("404 not found")
    elif init_state == 400:
        print("400 bad request")
    else:
        print("200 ok reload")
    

Request [237]: Content-Disposition: form-data; name="uploaded"; filename="shell.php"
Request general [237]: MixString-MixStringMixString-MixStringname=PureStringfilename=PureString.php
Selected Action [138]: 
../../hackable/uploads/payload.php succesfully uploaded!
